In [1]:
import mlrose
import random
import numpy as np
import time
import pandas as pd

# Functions

In [2]:
def random_hill_climb(problem, max_attempts=10, max_iters=np.inf, restarts=0,
                      init_state=None):
    """Use randomized hill climbing to find the optimum for a given
    optimization problem.
    Parameters
    ----------
    problem: optimization object
        Object containing fitness function optimization problem to be solved.
        For example, :code:`DiscreteOpt()`, :code:`ContinuousOpt()` or
        :code:`TSPOpt()`.
    max_attempts: int, default: 10
        Maximum number of attempts to find a better neighbor at each step.
    max_iters: int, default: np.inf
        Maximum number of iterations of the algorithm.
    restarts: int, default: 0
        Number of random restarts.
    init_state: array, default: None
        1-D Numpy array containing starting state for algorithm.
        If :code:`None`, then a random state is used.
    Returns
    -------
    best_state: array
        Numpy array containing state that optimizes the fitness function.
    best_fitness: float
        Value of fitness function at best state.
    References
    ----------
    Brownlee, J (2011). *Clever Algorithms: Nature-Inspired Programming
    Recipes*. `<http://www.cleveralgorithms.com>`_.
    """
    if (not isinstance(max_attempts, int) and not max_attempts.is_integer()) \
       or (max_attempts < 0):
        raise Exception("""max_attempts must be a positive integer.""")

    if (not isinstance(max_iters, int) and max_iters != np.inf
            and not max_iters.is_integer()) or (max_iters < 0):
        raise Exception("""max_iters must be a positive integer.""")

    if (not isinstance(restarts, int) and not restarts.is_integer()) \
       or (restarts < 0):
        raise Exception("""restarts must be a positive integer.""")

    if init_state is not None and len(init_state) != problem.get_length():
        raise Exception("""init_state must have same length as problem.""")

    best_fitness = -1*np.inf
    best_state = None

    best_iteration_data = []
    
    for _ in range(restarts + 1):
        # Initialize optimization problem and attempts counter
        if init_state is None:
            problem.reset()
        else:
            problem.set_state(init_state)

        attempts = 0
        iters = 0

        iteration_data = []
        while (attempts < max_attempts) and (iters < max_iters):
            iters += 1

            # Find random neighbor and evaluate fitness
            next_state = problem.random_neighbor()
            next_fitness = problem.eval_fitness(next_state)

            # If best neighbor is an improvement,
            # move to that state and reset attempts counter
            if next_fitness > problem.get_fitness():
                problem.set_state(next_state)
                attempts = 0

            else:
                attempts += 1
            
            iteration_data.append((iters, problem.get_fitness()))
            
        # Update best state and best fitness
        if problem.get_fitness() > best_fitness:
            best_fitness = problem.get_fitness()
            best_state = problem.get_state()
            
            best_iteration_data = iteration_data

    best_fitness = problem.get_maximize()*best_fitness
    return best_state, best_fitness, best_iteration_data

In [3]:
from mlrose.decay import GeomDecay
def simulated_annealing(problem, schedule=GeomDecay(), max_attempts=10,
                        max_iters=np.inf, init_state=None):
    """Use simulated annealing to find the optimum for a given
    optimization problem.
    Parameters
    ----------
    problem: optimization object
        Object containing fitness function optimization problem to be solved.
        For example, :code:`DiscreteOpt()`, :code:`ContinuousOpt()` or
        :code:`TSPOpt()`.
    schedule: schedule object, default: :code:`mlrose.GeomDecay()`
        Schedule used to determine the value of the temperature parameter.
    max_attempts: int, default: 10
        Maximum number of attempts to find a better neighbor at each step.
    max_iters: int, default: np.inf
        Maximum number of iterations of the algorithm.
    init_state: array, default: None
        1-D Numpy array containing starting state for algorithm.
        If :code:`None`, then a random state is used.
    Returns
    -------
    best_state: array
        Numpy array containing state that optimizes the fitness function.
    best_fitness: float
        Value of fitness function at best state.
    References
    ----------
    Russell, S. and P. Norvig (2010). *Artificial Intelligence: A Modern
    Approach*, 3rd edition. Prentice Hall, New Jersey, USA.
    """
    if (not isinstance(max_attempts, int) and not max_attempts.is_integer()) \
       or (max_attempts < 0):
        raise Exception("""max_attempts must be a positive integer.""")

    if (not isinstance(max_iters, int) and max_iters != np.inf
            and not max_iters.is_integer()) or (max_iters < 0):
        raise Exception("""max_iters must be a positive integer.""")

    if init_state is not None and len(init_state) != problem.get_length():
        raise Exception("""init_state must have same length as problem.""")

    # Initialize problem, time and attempts counter
    if init_state is None:
        problem.reset()
    else:
        problem.set_state(init_state)

    attempts = 0
    iters = 0
    
    iteration_data = []
    while (attempts < max_attempts) and (iters < max_iters):
        temp = schedule.evaluate(iters)
        iters += 1

        if temp == 0:
            break

        else:
            # Find random neighbor and evaluate fitness
            next_state = problem.random_neighbor()
            next_fitness = problem.eval_fitness(next_state)

            # Calculate delta E and change prob
            delta_e = next_fitness - problem.get_fitness()
            prob = np.exp(delta_e/temp)

            # If best neighbor is an improvement or random value is less
            # than prob, move to that state and reset attempts counter
            if (delta_e > 0) or (np.random.uniform() < prob):
                problem.set_state(next_state)
                attempts = 0

            else:
                attempts += 1
        iteration_data.append((iters, problem.get_fitness()))
        
    best_fitness = problem.get_maximize()*problem.get_fitness()
    best_state = problem.get_state()

    return best_state, best_fitness, iteration_data

In [4]:
def genetic_alg(problem, pop_size=200, mutation_prob=0.1, max_attempts=10,
                max_iters=np.inf):
    """Use a standard genetic algorithm to find the optimum for a given
    optimization problem.
    Parameters
    ----------
    problem: optimization object
        Object containing fitness function optimization problem to be solved.
        For example, :code:`DiscreteOpt()`, :code:`ContinuousOpt()` or
        :code:`TSPOpt()`.
    pop_size: int, default: 200
        Size of population to be used in genetic algorithm.
    mutation_prob: float, default: 0.1
        Probability of a mutation at each element of the state vector
        during reproduction, expressed as a value between 0 and 1.
    max_attempts: int, default: 10
        Maximum number of attempts to find a better state at each step.
    max_iters: int, default: np.inf
        Maximum number of iterations of the algorithm.
    Returns
    -------
    best_state: array
        Numpy array containing state that optimizes the fitness function.
    best_fitness: float
        Value of fitness function at best state.
    References
    ----------
    Russell, S. and P. Norvig (2010). *Artificial Intelligence: A Modern
    Approach*, 3rd edition. Prentice Hall, New Jersey, USA.
    """
    if pop_size < 0:
        raise Exception("""pop_size must be a positive integer.""")
    elif not isinstance(pop_size, int):
        if pop_size.is_integer():
            pop_size = int(pop_size)
        else:
            raise Exception("""pop_size must be a positive integer.""")

    if (mutation_prob < 0) or (mutation_prob > 1):
        raise Exception("""mutation_prob must be between 0 and 1.""")

    if (not isinstance(max_attempts, int) and not max_attempts.is_integer()) \
       or (max_attempts < 0):
        raise Exception("""max_attempts must be a positive integer.""")

    if (not isinstance(max_iters, int) and max_iters != np.inf
            and not max_iters.is_integer()) or (max_iters < 0):
        raise Exception("""max_iters must be a positive integer.""")

    # Initialize problem, population and attempts counter
    problem.reset()
    problem.random_pop(pop_size)
    attempts = 0
    iters = 0

    iteration_data = []
    while (attempts < max_attempts) and (iters < max_iters):
        iters += 1

        # Calculate breeding probabilities
        problem.eval_mate_probs()

        # Create next generation of population
        next_gen = []

        for _ in range(pop_size):
            # Select parents
            selected = np.random.choice(pop_size, size=2,
                                        p=problem.get_mate_probs())
            parent_1 = problem.get_population()[selected[0]]
            parent_2 = problem.get_population()[selected[1]]

            # Create offspring
            child = problem.reproduce(parent_1, parent_2, mutation_prob)
            next_gen.append(child)

        next_gen = np.array(next_gen)
        problem.set_population(next_gen)

        next_state = problem.best_child()
        next_fitness = problem.eval_fitness(next_state)

        # If best child is an improvement,
        # move to that state and reset attempts counter
        if next_fitness > problem.get_fitness():
            problem.set_state(next_state)
            attempts = 0

        else:
            attempts += 1
            
        iteration_data.append((iters, problem.get_fitness()))

    best_fitness = problem.get_maximize()*problem.get_fitness()
    best_state = problem.get_state()

    return best_state, best_fitness, iteration_data

In [5]:
def mimic(problem, pop_size=200, keep_pct=0.2, max_attempts=10,
          max_iters=np.inf):
    """Use MIMIC to find the optimum for a given optimization problem.
    Parameters
    ----------
    problem: optimization object
        Object containing fitness function optimization problem to be solved.
        For example, :code:`DiscreteOpt()` or :code:`TSPOpt()`.
    pop_size: int, default: 200
        Size of population to be used in algorithm.
    keep_pct: float, default: 0.2
        Proportion of samples to keep at each iteration of the algorithm,
        expressed as a value between 0 and 1.
    max_attempts: int, default: 10
        Maximum number of attempts to find a better neighbor at each step.
    max_iters: int, default: np.inf
        Maximum number of iterations of the algorithm.
    Returns
    -------
    best_state: array
        Numpy array containing state that optimizes the fitness function.
    best_fitness: float
        Value of fitness function at best state.
    References
    ----------
    De Bonet, J., C. Isbell, and P. Viola (1997). MIMIC: Finding Optima by
    Estimating Probability Densities. In *Advances in Neural Information
    Processing Systems* (NIPS) 9, pp. 424–430.
    Note
    ----
    MIMIC cannot be used for solving continuous-state optimization problems.
    """
    if problem.get_prob_type() == 'continuous':
        raise Exception("""problem type must be discrete or tsp.""")

    if pop_size < 0:
        raise Exception("""pop_size must be a positive integer.""")
    elif not isinstance(pop_size, int):
        if pop_size.is_integer():
            pop_size = int(pop_size)
        else:
            raise Exception("""pop_size must be a positive integer.""")

    if (keep_pct < 0) or (keep_pct > 1):
        raise Exception("""keep_pct must be between 0 and 1.""")

    if (not isinstance(max_attempts, int) and not max_attempts.is_integer()) \
       or (max_attempts < 0):
        raise Exception("""max_attempts must be a positive integer.""")

    if (not isinstance(max_iters, int) and max_iters != np.inf
            and not max_iters.is_integer()) or (max_iters < 0):
        raise Exception("""max_iters must be a positive integer.""")

    # Initialize problem, population and attempts counter
    problem.reset()
    problem.random_pop(pop_size)
    attempts = 0
    iters = 0

    iteration_data = []
    while (attempts < max_attempts) and (iters < max_iters):
        iters += 1

        # Get top n percent of population
        problem.find_top_pct(keep_pct)

        # Update probability estimates
        problem.eval_node_probs()

        # Generate new sample
        new_sample = problem.sample_pop(pop_size)
        problem.set_population(new_sample)

        next_state = problem.best_child()

        next_fitness = problem.eval_fitness(next_state)

        # If best child is an improvement,
        # move to that state and reset attempts counter
        if next_fitness > problem.get_fitness():
            problem.set_state(next_state)
            attempts = 0

        else:
            attempts += 1
            
        iteration_data.append((iters, problem.get_fitness()))

    best_fitness = problem.get_maximize()*problem.get_fitness()
    best_state = problem.get_state().astype(int)

    return best_state, best_fitness, iteration_data

# Flip Flop - Simulated Annealing

In [6]:
#defining the fitness function
fitness_func = mlrose.FlipFlop()

#define the problem
problem_fit = mlrose.DiscreteOpt(200, fitness_fn = fitness_func, maximize = True, max_val = 2)

<b>Random Hill Climb</b>

In [7]:
#generate iteration graph

current_best_fitness = None


startFirst = time.time()
bestHillState, bestHillFitness, iteration_data = random_hill_climb(problem_fit, restarts = 50)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingIterationsff.csv")


#generate numrestarts hyperparameter graph
restart_data = []
for num_restarts in range (0, 51):
    new_state, new_fitness = mlrose.random_hill_climb(problem_fit, restarts = num_restarts)
    restart_data.append((num_restarts, new_fitness))
#save hyperparameter data to a file    
num_restarts, fitness = [i[0] for i in restart_data], [i[1] for i in restart_data]
plotData = [('Number of Restarts', num_restarts), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingNumRestartsff.csv")

#graph for Iterations size vs time
time_data = []
for numFlipFlop in range(20, 200, 20):
    problem_fit_test = mlrose.DiscreteOpt(numFlipFlop, fitness_fn = fitness_func, maximize = True, max_val = 2)
    start = time.time()
    bestmimicIterationState, bestmimicIterationFitness = mlrose.random_hill_climb(problem_fit_test, restarts = 50)
    end = time.time()
    time_data.append((numFlipFlop, end-start))
numFlipFlop, times = [i[0] for i in time_data], [i[1] for i in time_data]
plotData = [('Flip Flop Problem Size', numFlipFlop), ('Time', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbTimingff.csv")

#print out time taken
print("time taken is:")
print(endFirst-startFirst)

time taken is:
0.36264634132385254


<b>Simulated Annealing</b>

In [8]:
current_best_fitness = None


startFirst = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = simulated_annealing(problem_fit)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("simulatedAnnealingIterationsff.csv")

#graph for Iterations size vs time
time_data = []
for numFlipFlop in range(20, 200, 20):
    problem_fit_test = mlrose.DiscreteOpt(numFlipFlop, fitness_fn = fitness_func, maximize = True, max_val = 2)
    start = time.time()
    bestmimicIterationState, bestmimicIterationFitness = mlrose.simulated_annealing(problem_fit_test)
    end = time.time()
    time_data.append((numFlipFlop, end-start))
numFlipFlop, times = [i[0] for i in time_data], [i[1] for i in time_data]
plotData = [('Flip Flop Problem Size', numFlipFlop), ('Time', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("SimulatedAnnealingTimingff.csv")


#print out time taken
print("time taken is:")
print(endFirst-startFirst)

time taken is:
0.1429884433746338


<b>Genetic Algorithm</b>

In [ ]:
current_best_fitness = None


startFirst = time.time()
bestGeneticState, bestGeneticFitness, iteration_data = genetic_alg(problem_fit, pop_size = 200)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAIterationsff.csv")

time_data = []
#graph for population size vs time
for popSize in range(10, 310, 10):
    start = time.time()
    bestGeneticPopState, bestGeneticPopFitness = mlrose.genetic_alg(problem_fit, pop_size = popSize)
    end = time.time()
    time_data.append((popSize, bestGeneticPopFitness, end-start))
pop_size, fitness, times = [i[0] for i in time_data], [i[1] for i in time_data], [i[2] for i in time_data]
plotData = [('Population Size', pop_size), ('Fitness', fitness), ('Time to Converge', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAHyperParameterff.csv")
    
#graph for Iterations size vs time
time_data = []
for numFlipFlop in range(20, 200, 20):
    problem_fit_test = mlrose.DiscreteOpt(numFlipFlop, fitness_fn = fitness_func, maximize = True, max_val = 2)
    start = time.time()
    bestmimicIterationState, bestmimicIterationFitness = mlrose.genetic_alg(problem_fit_test)
    end = time.time()
    time_data.append((numFlipFlop, end-start))
numFlipFlop, times = [i[0] for i in time_data], [i[1] for i in time_data]
plotData = [('Flip Flop Problem Size', numFlipFlop), ('Time', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GATimingff.csv")    
    
#print out time taken
print("time taken is:")
print(endFirst-startFirst)

time taken is:
1.56785249710083


<b>Mimic</b>

In [ ]:
current_best_fitness = None


start = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = mimic(problem_fit, max_iters = 30)
end = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("mimicIterationsff.csv")


#graph for Iterations size vs time
time_data = []
for numFlipFlop in range(20, 200, 20):
    problem_fit_test = mlrose.DiscreteOpt(numFlipFlop, fitness_fn = fitness_func, maximize = True, max_val = 2)
    start = time.time()
    bestmimicIterationState, bestmimicIterationFitness = mlrose.mimic(problem_fit_test, max_iters = 25)
    end = time.time()
    time_data.append((numFlipFlop, end-start))
numFlipFlop, times = [i[0] for i in time_data], [i[1] for i in time_data]
plotData = [('Flip Flop Problem Size', numFlipFlop), ('Time', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("MimicTimingff.csv")

#print out time taken
print("time taken is:")
print(end-start)

# Knapsack - Genetic

In [ ]:
#defining the fitness function
weights = [random.randint(1, 60) for i in range(50)]
values = [random.randint(1, 60) for i in range(50)]
max_weight_pct = 0.6
fitness_func = mlrose.Knapsack(weights, values, max_weight_pct)

#define the problem
problem_fit = mlrose.DiscreteOpt(50, fitness_fn = fitness_func, maximize = True, max_val = 3)
problem_fit_hillClimb = mlrose.DiscreteOpt(50, fitness_fn = fitness_func, maximize = True, max_val = 2)

<b>Random Hill Climb</b>

In [ ]:
#generate iteration graph

current_best_fitness = None


startFirst = time.time()
bestHillState, bestHillFitness, iteration_data = random_hill_climb(problem_fit_hillClimb, restarts = 50)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingIterationskn.csv")


#generate numrestarts hyperparameter graph
restart_data = []
for num_restarts in range (0, 51):
    new_state, new_fitness = mlrose.random_hill_climb(problem_fit_hillClimb, restarts = num_restarts)
    restart_data.append((num_restarts, new_fitness))
#save hyperparameter data to a file    
num_restarts, fitness = [i[0] for i in restart_data], [i[1] for i in restart_data]
plotData = [('Number of Restarts', num_restarts), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingNumRestartskn.csv")

#print out time taken
print("time taken is:")
print(endFirst-startFirst)

<b>Simulated Annealing</b>

In [ ]:
current_best_fitness = None


startFirst = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = simulated_annealing(problem_fit)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("simulatedAnnealingIterationskn.csv")


#print out time taken
print("time taken is:")
print(endFirst-startFirst)

<b>Genetic Algorithm</b>

In [ ]:
current_best_fitness = None


startFirst = time.time()
bestGeneticState, bestGeneticFitness, iteration_data = genetic_alg(problem_fit, pop_size = 200)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAIterationskn.csv")

time_data = []
#graph for population size vs time
for popSize in range(10, 310, 10):
    start = time.time()
    bestGeneticPopState, bestGeneticPopFitness = mlrose.genetic_alg(problem_fit, pop_size = popSize)
    end = time.time()
    time_data.append((popSize, bestGeneticPopFitness, end-start))
pop_size, fitness, times = [i[0] for i in time_data], [i[1] for i in time_data], [i[2] for i in time_data]
plotData = [('Population Size', pop_size), ('Fitness', fitness), ('Time to Converge', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAHyperParameterkn.csv")
    
    
#print out time taken
print("time taken is:")
print(endFirst-startFirst)

<b>Mimic</b>

In [ ]:
current_best_fitness = None


start = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = mimic(problem_fit, max_iters = 30)
end = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("mimicIterationskn.csv")

#print out time taken
print("time taken is:")
print(end-start)

# N-Queens

In [6]:
# Define alternative N-Queens fitness function for maximization problem
def queens_max(state):

   # Initialize counter
    fitness_cnt = 0
      # For all pairs of queens
    for i in range(len(state) - 1):
        for j in range(i + 1, len(state)):

            # Check for horizontal, diagonal-up and diagonal-down attacks
            if (state[j] != state[i]) \
                and (state[j] != state[i] + (j - i)) \
                and (state[j] != state[i] - (j - i)):

               # If no attacks, then increment counter
               fitness_cnt += 1

    return fitness_cnt

In [7]:
#defining the fitness function
fitness_func = mlrose.CustomFitness(queens_max)

#define the problem
problem_fit = mlrose.DiscreteOpt(120, fitness_fn = fitness_func, maximize = True, max_val = 120)

<b>Random Hill Climb</b>

In [14]:
#generate iteration graph

current_best_fitness = None


startFirst = time.time()
bestHillState, bestHillFitness, iteration_data = random_hill_climb(problem_fit, restarts = 20, max_attempts = 30)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingIterationsQ.csv")


#generate numrestarts hyperparameter graph
restart_data = []
for num_restarts in range (0, 21):
    new_state, new_fitness = mlrose.random_hill_climb(problem_fit, restarts = num_restarts)
    restart_data.append((num_restarts, new_fitness))
#save hyperparameter data to a file    
num_restarts, fitness = [i[0] for i in restart_data], [i[1] for i in restart_data]
plotData = [('Number of Restarts', num_restarts), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingNumRestartsQ.csv")

#print out time taken
print("time taken is:")
print(endFirst-startFirst)

time taken is:
79.21764135360718


<b>Simulated Annealing</b>

In [12]:
current_best_fitness = None


startFirst = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = simulated_annealing(problem_fit, max_attempts = 30)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("simulatedAnnealingIterationsQ.csv")


#print out time taken
print("time taken is:")
print(endFirst-startFirst)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: overflow encountered in exp


time taken is:
11.94499397277832


<b>Genetic Algorithm</b>

In [13]:
current_best_fitness = None


startFirst = time.time()
bestGeneticState, bestGeneticFitness, iteration_data = genetic_alg(problem_fit, max_attempts = 30, pop_size = 200)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAIterationsQ.csv")

time_data = []
#graph for population size vs time
for popSize in range(10, 310, 10):
    start = time.time()
    bestGeneticPopState, bestGeneticPopFitness = mlrose.genetic_alg(problem_fit, pop_size = popSize)
    end = time.time()
    time_data.append((popSize, bestGeneticPopFitness, end-start))
pop_size, fitness, times = [i[0] for i in time_data], [i[1] for i in time_data], [i[2] for i in time_data]
plotData = [('Population Size', pop_size), ('Fitness', fitness), ('Time to Converge', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAHyperParameterQ.csv")
    
    
#print out time taken
print("time taken is:")
print(endFirst-startFirst)

KeyboardInterrupt: 

<b>Mimic</b>

In [11]:
current_best_fitness = None


start = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = mimic(problem_fit, max_iters = 30)
end = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("mimicIterationsQ.csv")

#print out time taken
print("time taken is:")
print(end-start)

time taken is:
100.07028436660767


# MaxKColoring

In [12]:
def evaluateKMax(state, edges):
        """Evaluate the fitness of a state vector.

        Parameters
        ----------
        state: array
            State array for evaluation.

        Returns
        -------
        fitness: float
            Value of fitness function.
        """

        fitness = 0

        for i in range(len(edges)):
            # Check for adjacent nodes of the same color
            if state[edges[i][0]] != state[edges[i][1]]:
                fitness += 1

        return fitness

In [13]:
NumNodes = 100
K = 3

numEdges = int(.3 * NumNodes * NumNodes)
completeGraph = [(i, j) for i in range(NumNodes) for j in range(NumNodes)]
edges = random.sample(completeGraph, numEdges)

In [14]:
fitness_cust = mlrose.CustomFitness(evaluateKMax, problem_type = 'discrete', edges = edges)
problem_fit = mlrose.DiscreteOpt(NumNodes, fitness_fn = fitness_cust, maximize = True, max_val = K)

<b>Random Hill Climb</b>

In [15]:
#generate iteration graph

current_best_fitness = None


startFirst = time.time()
bestHillState, bestHillFitness, iteration_data = random_hill_climb(problem_fit, restarts = 50)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingIterationsMaxK.csv")


#generate numrestarts hyperparameter graph
restart_data = []
for num_restarts in range (0, 51):
    new_state, new_fitness = mlrose.random_hill_climb(problem_fit, restarts = num_restarts)
    restart_data.append((num_restarts, new_fitness))
#save hyperparameter data to a file    
num_restarts, fitness = [i[0] for i in restart_data], [i[1] for i in restart_data]
plotData = [('Number of Restarts', num_restarts), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingNumRestartsMaxK.csv")

#print out time taken
print("time taken is:")
print(endFirst-startFirst)

time taken is:
7.481658458709717


<b>Simulated Annealing</b>

In [16]:
current_best_fitness = None


startFirst = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = simulated_annealing(problem_fit)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("simulatedAnnealingIterationsMaxK.csv")


#print out time taken
print("time taken is:")
print(endFirst-startFirst)

time taken is:
0.3621673583984375


<b>Genetic Algorithm</b>

In [17]:
current_best_fitness = None


startFirst = time.time()
bestGeneticState, bestGeneticFitness, iteration_data = genetic_alg(problem_fit, pop_size = 200)
endFirst = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAIterationsMaxK.csv")

time_data = []
#graph for population size vs time
for popSize in range(10, 310, 10):
    start = time.time()
    bestGeneticPopState, bestGeneticPopFitness = mlrose.genetic_alg(problem_fit, pop_size = popSize)
    end = time.time()
    time_data.append((popSize, bestGeneticPopFitness, end-start))
pop_size, fitness, times = [i[0] for i in time_data], [i[1] for i in time_data], [i[2] for i in time_data]
plotData = [('Population Size', pop_size), ('Fitness', fitness), ('Time to Converge', times)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("GAHyperParameterMaxK.csv")
    
    
#print out time taken
print("time taken is:")
print(endFirst-startFirst)

time taken is:
4.133784055709839


<b>Mimic</b>

In [18]:
current_best_fitness = None


start = time.time()
bestSimulatedState, bestSimulatedFitness, iteration_data = mimic(problem_fit, max_iters = 30)
end = time.time()

#save iteration data to a file    
iteration_num, fitness = [i[0] for i in iteration_data], [i[1] for i in iteration_data]
plotData = [('Iteration Number', iteration_num), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("mimicIterationsMaxK.csv")

#print out time taken
print("time taken is:")
print(end-start)

time taken is:
104.66429328918457
